In [1]:
!date

Wed May  1 22:05:01 CEST 2024


<P>Can Europe’s renewable incentive policies hedge against the
geopolitical risk of China’s supply chain dominance in the solar pv
industry? Have incentive policies an impact on the diversity of
suppliers and import dependency?</P>

<P>To answer the main research question:
Panel data regression with China's import share (=indicator for
geopolitical risk) as the dependent variable and Feed-in
tariffs/European incentive policies as the independent variable. And
some control variables( might be too many)
</P>

    Main regression:
<math>
    $$y = b + w_1x_1 + w_2x_2 + w_3x_3 + ...+w_m x_m $$
</math>

In [22]:
# general imports
import numpy as np
import pandas as pd
from scipy.stats import chi2

import pathlib

## Load Data

<P>The data is from 2005 until 2019, for 24 EU countries (so EU without
Malta, Cyprus and Luxembourg)</P>

In [3]:
myfile = pathlib.Path('../datasets/panel_regression_data.csv')

In [4]:
data = pd.read_csv(myfile)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Country                                 360 non-null    object 
 1   Year                                    360 non-null    int64  
 2   Import Share                            360 non-null    float64
 3   Fit                                     360 non-null    float64
 4   GDP/capita                              360 non-null    float64
 5   Energy Consumption                      360 non-null    float64
 6   Annual Solar Capacity Addition          360 non-null    float64
 7   TechAdvancement                         360 non-null    int64  
 8   Tech Advancement CN                     360 non-null    int64  
 9   Trade Policies EU                       360 non-null    float64
 10  Fixed Asset Investment CN (prior year)  360 non-null    float6

In [5]:
data.head(5)

,Country,Year,Import Share,Fit,GDP/capita,Energy Consumption,Annual Solar Capacity Addition,TechAdvancement,Tech Advancement CN,Trade Policies EU,Fixed Asset Investment CN (prior year),Avg Wage Difference,Environ. St. Difference
0,Austria,2005,0.86,0.672,38417.46,425.46,-6.0,104,573,0.0,525.7,2346.470000,1.6111
1,Austria,2006,15.73,0.678,40669.33,421.01,1.4,103,979,0.0,771.5,2408.950000,1.8333
2,Austria,2007,7.33,0.521,46915.34,411.99,1.9,117,1621,0.0,1110.7,2407.100000,1.5556
3,Austria,2008,6.67,0.559,51919.98,420.01,5.9,123,2364,0.0,1608.4,2568.188377,1.8611
4,Austria,2009,6.99,0.530,48153.32,406.84,18.8,84,4149,0.0,2344.0,2974.780000,1.9722


In [6]:
# collect independent variables
dvar = data.columns[3:].to_list()
print(dvar)

['Fit', 'GDP/capita', 'Energy Consumption', 'Annual Solar Capacity Addition', 'TechAdvancement', 'Tech Advancement CN', 'Trade Policies EU', 'Fixed Asset Investment CN (prior year)', 'Avg Wage Difference', 'Environ. St. Difference']


## Ordinary Least Squares (OLS) regression
Uni-variate simple linear regression

_CN’s share in
ImportEUit_=_β_0​+_β_1​_FITEUit_​+_β_2​_GDPprocapitaEUit_​+_β_3​_RenewableEnergyConsumptionEUit_​+_β_4​_SolarPVcapacityEUit_​+_β_5​_TechAdvancementEUit_​+_
β_6_TechAdvancementCNit +β_7_TradePoliciesEUt_​+_
__β8_FixedAssetInvestmentCN_t-1_​+_
β_9​​_(DifferenceinAvgWagesEUCNit)+
β_10_(DifferenceinEnvironmentalSt.EUCNit)+ui_​+_ϵit_

In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [8]:
# Simple linear regression with ordinary least squares in statmodels

#x = data.loc[:,[dvar]] # independent variables
X = data[dvar]
y = data['Import Share'] # dependent variable

## fit a OLS model with y = b + w*X
x = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Import Share   R-squared (uncentered):                   0.746
Model:                            OLS   Adj. R-squared (uncentered):              0.738
Method:                 Least Squares   F-statistic:                              102.5
Date:                Wed, 01 May 2024   Prob (F-statistic):                    1.27e-97
Time:                        22:05:04   Log-Likelihood:                         -1400.5
No. Observations:                 360   AIC:                                      2821.
Df Residuals:                     350   BIC:                                      2860.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Fit                                       14.7021      3.136      4.688      0.000       8.534      20.870
GDP/capita                                -0.0001   7.82e-05     -1.690      0.092      -0.000    2.17e-05
Energy Consumption                        -0.0004      0.001     -0.355      0.723      -0.003       0.002
Annual Solar Capacity Addition             0.0023      0.001      2.462      0.014       0.000       0.004
TechAdvancement                            0.0128      0.008      1.621      0.106      -0.003       0.028
Tech Advancement CN                       -0.0012      0.000     -4.522      0.000      -0.002      -0.001
Trade Policies EU                         -0.1598      0.042     -3.817      0.000      -0.242      -0.077
Fixed Asset Investment CN (prior year)     0.0034      0.000      7.725      0.000       0.003       0.004
Avg Wage Difference                        0.0040      0.001      3.511      0.001       0.002       0.006
Environ. St. Difference                    2.6012      0.791      3.289      0.001       1.046       4.157
==============================================================================
Omnibus:                       81.793   Durbin-Watson:                   1.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              190.874
Skew:                           1.125   Prob(JB):                     3.57e-42
Kurtosis:                       5.768   Cond. No.                     1.80e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Fixed Effects model

In [9]:
from linearmodels.panel import PanelOLS

In [10]:
# we need multi-index first
data = data.set_index(['Country', 'Year'])
X = data[dvar]
y = data['Import Share'] # dependent variable

# Fixed Effects Model 
fe_model = PanelOLS(y, X, entity_effects=True) # entity_effects=True for offset
fe_results = fe_model.fit()
fe_results

Dep. Variable:,Import Share,R-squared:,0.2884
Estimator:,PanelOLS,R-squared (Between):,0.2483
No. Observations:,360,R-squared (Within):,0.2884
Date:,"Wed, May 01 2024",R-squared (Overall):,0.2592
Time:,22:05:04,Log-likelihood,-1351.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,13.210
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(10,326)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,13.210


## Random Effects Model

In [13]:
from linearmodels.panel import RandomEffects

In [14]:
re_model = RandomEffects(y, X)
re_results = re_model.fit()
re_results

Dep. Variable:,Import Share,R-squared:,0.5215
Estimator:,RandomEffects,R-squared (Between):,0.9127
No. Observations:,360,R-squared (Within):,0.2836
Date:,"Wed, May 01 2024",R-squared (Overall):,0.7413
Time:,22:06:31,Log-likelihood,-1365.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,38.140
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(10,350)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,38.140


## Two-Stage Least Squares (2SLS) Regression Analysis

In [48]:
from linearmodels.iv import IV2SLS

In [56]:
dep = data['Import Share'] 
exog = None
endog = data[dvar[:3]] # test the first two variables
instr = data[dvar[3:]]

resultIV = IV2SLS(dep, exog, endog, instr).fit() 

resultIV.summary 

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:           Import Share   R-squared:                      0.6198
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6166
No. Observations:                 360   F-statistic:                    597.82
Date:                Wed, May 01 2024   P-value (F-stat)                0.0000
Time:                        23:03:18   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                  
======================================================================================
                    Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------
Fit                    14.036     7.2725     1.9301     0.0536     -0.2174      28.290
GDP/capita             0.0004  3.725e-05     9.7157     0.0000      0.0003      0.0004
Energy Consumption     0.0052     0.0013     3.9620     0.0001      0.0026      0.0078
======================================================================================

Endogenous: Fit, GDP/capita, Energy Consumption
Instruments: Annual Solar Capacity Addition, TechAdvancement, Tech Advancement CN, Trade Policies EU, Fixed Asset Investment CN (prior year), Avg Wage Difference, Environ. St. Difference
Robust Covariance (Heteroskedastic)
Debiased: False
"""

In [57]:
resultIV.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 13.4133
P-value: 0.0000
Distributed: F(3,354)
WaldTestStatistic, id: 0x130968da0

## Comparing

In [58]:
from linearmodels.panel import compare

In [59]:
hausman_results = compare({"Fixed Effects": fe_results, "Random Effects": re_results}, precision='tstats')
print(hausman_results)


                               Model Comparison                              
                                              Fixed Effects    Random Effects
-----------------------------------------------------------------------------
Dep. Variable                                  Import Share      Import Share
Estimator                                          PanelOLS     RandomEffects
No. Observations                                        360               360
Cov. Est.                                        Unadjusted        Unadjusted
R-squared                                            0.2884            0.5215
R-Squared (Within)                                   0.2884            0.2836
R-Squared (Between)                                  0.2483            0.9127
R-Squared (Overall)                                  0.2592            0.7413
F-statistic                                          13.210            38.140
P-value (F-stat)                                     0.0000     